# K리그 경기 변곡점 분석 - 샘플 데이터

이 노트북은 샘플 데이터를 사용하여 경기 변곡점을 분석합니다.

In [ ]:
# 환경 설정 및 라이브러리 import
import sys
from pathlib import Path
from datetime import datetime
import random
import os
from IPython.display import Image, display

# 프로젝트 루트를 경로에 추가
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# 프로젝트 모듈 import
from src.data.models import MatchData, MatchEvent
from src.analysis.turning_point import detect_turning_points
from src.explanation.generator import ExplanationGenerator
from src.visualization.plotter import plot_momentum_curve

print("라이브러리 로드 완료!")

## 1. 샘플 데이터 생성

In [ ]:
# 샘플 경기 데이터 생성
def create_sample_match_data() -> MatchData:
    """테스트용 샘플 경기 데이터 - 변곡점이 명확한 시나리오"""
    events = []
    random.seed(42)  # 재현 가능한 결과
    
    # 전반 초반 (0-20분): 홈팀 우세
    for minute in range(0, 20):
        events.append(MatchEvent(
            minute=minute,
            team="홈팀",
            event_type="shot",
            x=75.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-10, 10),
            success=True,
            xg=0.15 + random.uniform(-0.05, 0.05)
        ))
        events.append(MatchEvent(
            minute=minute,
            team="홈팀",
            event_type="pass",
            x=60.0 + random.uniform(-5, 5),
            y=45.0 + random.uniform(-5, 5),
            success=True
        ))
        events.append(MatchEvent(
            minute=minute,
            team="원정팀",
            event_type="defense",
            x=30.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-5, 5),
            success=True
        ))
    
    # 전반 중반 (20-45분): 원정팀 반격 - 변곡점 1
    for minute in range(20, 45):
        events.append(MatchEvent(
            minute=minute,
            team="원정팀",
            event_type="shot",
            x=25.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-10, 10),
            success=True,
            xg=0.12 + random.uniform(-0.05, 0.05)
        ))
        events.append(MatchEvent(
            minute=minute,
            team="원정팀",
            event_type="pass",
            x=40.0 + random.uniform(-5, 5),
            y=55.0 + random.uniform(-5, 5),
            success=True
        ))
        if minute % 3 == 0:
            events.append(MatchEvent(
                minute=minute,
                team="홈팀",
                event_type="shot",
                x=70.0 + random.uniform(-5, 5),
                y=50.0 + random.uniform(-10, 10),
                success=True,
                xg=0.10 + random.uniform(-0.03, 0.03)
            ))
    
    # 후반 초반 (45-70분): 홈팀 재반격 - 변곡점 2
    for minute in range(45, 70):
        events.append(MatchEvent(
            minute=minute,
            team="홈팀",
            event_type="shot",
            x=80.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-10, 10),
            success=True,
            xg=0.20 + random.uniform(-0.05, 0.05)
        ))
        events.append(MatchEvent(
            minute=minute,
            team="홈팀",
            event_type="pass",
            x=65.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-5, 5),
            success=True
        ))
        events.append(MatchEvent(
            minute=minute,
            team="홈팀",
            event_type="defense",
            x=45.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-5, 5),
            success=True
        ))
    
    # 후반 후반 (70-90분): 원정팀 최종 공격 - 변곡점 3
    for minute in range(70, 90):
        events.append(MatchEvent(
            minute=minute,
            team="원정팀",
            event_type="shot",
            x=20.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-10, 10),
            success=True,
            xg=0.18 + random.uniform(-0.05, 0.05)
        ))
        events.append(MatchEvent(
            minute=minute,
            team="원정팀",
            event_type="pass",
            x=35.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-5, 5),
            success=True
        ))
        events.append(MatchEvent(
            minute=minute,
            team="홈팀",
            event_type="pass",
            x=55.0 + random.uniform(-5, 5),
            y=50.0 + random.uniform(-5, 5),
            success=random.choice([True, False, False])
        ))
    
    return MatchData(
        match_id="sample_001",
        home_team="서울 FC",
        away_team="수원 삼성",
        match_date=datetime.now(),
        events=events,
        final_score={"home": 2, "away": 1}
    )

# 샘플 데이터 생성
match_data = create_sample_match_data()
print(f"경기: {match_data.home_team} vs {match_data.away_team}")
print(f"이벤트 수: {len(match_data.events)}")
print(f"최종 스코어: {match_data.final_score['home']} - {match_data.final_score['away']}")

## 2. 변곡점 탐지

In [ ]:
# 변곡점 탐지
print("변곡점 탐지 중...")
turning_points = detect_turning_points(match_data)
print(f"탐지된 변곡점: {len(turning_points)}개\n")

# 변곡점 정보 출력
for tp in turning_points:
    team_name = (
        match_data.home_team if tp.team_advantage == 'home'
        else match_data.away_team
    )
    print(f"[{tp.minute}분] {team_name}")
    print(f"  유형: {tp.change_type}")
    print(f"  지표: {', '.join(tp.indicators)}")
    print()

## 3. 설명 생성

In [ ]:
# 설명 생성
explanation_gen = ExplanationGenerator()

for tp in turning_points:
    team_name = (
        match_data.home_team if tp.team_advantage == 'home'
        else match_data.away_team
    )
    tp.explanation = explanation_gen.generate_explanation(tp, team_name)
    print(f"[{tp.minute}분] {team_name}")
    print(f"  유형: {tp.change_type}")
    print(f"  지표: {', '.join(tp.indicators)}")
    print(f"  설명: {tp.explanation}\n")

# 전체 요약
summary = explanation_gen.generate_summary(
    turning_points,
    match_data.home_team,
    match_data.away_team
)
print(f"전체 요약:")
print(f"  {summary}")

## 4. 시각화

In [ ]:
# 모멘텀 곡선 그래프 생성
print("그래프 생성 중...")
output_path = "momentum_curve_sample.png"
plot_momentum_curve(match_data, turning_points, output_path)
print(f"그래프가 '{output_path}'로 저장되었습니다.\n")

# 이미지를 노트북 셀에 인라인으로 표시
if os.path.exists(output_path):
    display(Image(output_path))